# Post IMARIS table merge (For All-Spots)

* Merges all the statistical tables from the different IMARIS files
* Creates a main table for one whole experiment

* Plot multiple graphs of the data

### Loading the modules

In [ ]:
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import seaborn as sns
import numpy as np
import os
from tkinter import filedialog
import click
from scipy.stats import mannwhitneyu
from sklearn.mixture import GaussianMixture
from scipy.stats import norm

Identification of the main folder and of the experiment name

In [ ]:
# Select directory
ExpFolder = filedialog.askdirectory(title='Select the directory containing all the IMARIS folders')

# List all the subfolders
FolderList = [f for f in os.listdir(ExpFolder) if os.path.isdir(os.path.join(ExpFolder, f))]

Final_Table = pd.DataFrame()

### Generation of the final table
Genetration of a table to associate to a foci :
- the distance to the nucloid surface inside
- the distance to the nucleoid surface outside
- the intensity (in the Red channel) of the foci 

Merging all the table within an experiment

In [ ]:
# Loop in each folder / subfolders to find the tables
for Folder in FolderList:
    FolderPath = os.path.join(ExpFolder, Folder)
    SubFolderList = os.listdir(FolderPath)
    for SubFolder in SubFolderList:
        Table = 0
        if SubFolder.endswith('Statistics'):
            SubFolderPath = os.path.join(FolderPath,SubFolder)
            FileList = os.listdir(SubFolderPath)
            # Open the three tables in pd DataFrames
            for File in FileList:
                if File.endswith("Intensity_Mean_Ch=3_Img=1.csv"):
                    FilePath = os.path.join(SubFolderPath,File)
                    Spot_Int_RED_Table_Single = pd.read_csv(FilePath,skiprows=3)
                elif File.endswith("Intensity_Center_Ch=4_Img=1.csv"):
                    FilePath = os.path.join(SubFolderPath,File)
                    Spot_Distance_Inside_Table_Single = pd.read_csv(FilePath,skiprows=3)
                elif File.endswith("Intensity_Center_Ch=5_Img=1.csv"):
                    FilePath = os.path.join(SubFolderPath,File)
                    Spot_Distance_Outside_Table_Single = pd.read_csv(FilePath,skiprows=3)
            # Prepare the final table
            Final_Table_Single = pd.DataFrame().assign(Red_Channel_Mean_Intensity=Spot_Int_RED_Table_Single['Intensity Mean'], Distance_Inside=Spot_Distance_Inside_Table_Single['Intensity Center'],Distance_Outside=Spot_Distance_Outside_Table_Single['Intensity Center'])      
            Table = 1
        if Table==1:
            Final_Table = pd.concat([Final_Table,Final_Table_Single])

Final_Table["Total_Distance"]=Final_Table["Distance_Inside"]+(-Final_Table["Distance_Outside"])

# Save it as a CSV
Final_Table.to_csv(ExpFolder+'/All_Data.csv')

Additional filtering of the Outside Spots 
Removing the aberant spots (further than 0.2 µm from the nucleoide surface)

In [ ]:
Final_Table = Final_Table[Final_Table.Distance_Outside < 0.2]

### Graphical output

In [ ]:
Int = Final_Table.iloc[:,0].astype('float64')
Dist = Final_Table.iloc[:,3].astype('float64')

sns.set_context("paper")
ax = sns.scatterplot(x=Int, y=Dist,
                hue=Int,
                legend=False,
                palette=sns.dark_palette("red", reverse=False, as_cmap=True))
ax.set_xlabel("Level of pathogenicity island expression (mean intensity)")
ax.set_ylabel("Foci distance to nucleoid surface (µm)")
ax.set_title("Foci localisation upon pathogenicity island expression",)

# Save fig as PDF
plt.savefig(ExpFolder+"/DotPlot_All-Foci.pdf")

# Save fig as PNG
plt.savefig(ExpFolder+"/DotPlot_All-Foci.png", dpi=300)

### Finding the threshold

Estimation of the limit of the Negative population by Gaussian mixing approach

In [ ]:
# Get a ndArray
RedCh = Final_Table['Red_Channel_Mean_Intensity']
RedCh = RedCh.to_numpy()
RedCh = RedCh.reshape((RedCh.shape[0], 1))

# Process Gaussian mixing (2 components)
def mix_pdf(x, loc, scale, weights):
    d = np.zeros_like(x)
    for mu, sigma, pi in zip(loc, scale, weights):
        d += pi * norm.pdf(x, loc=mu, scale=sigma)
    return d

mix = GaussianMixture(n_components=2, random_state=1, max_iter=100).fit(RedCh)
pi, mu, sigma = mix.weights_.flatten(), mix.means_.flatten(), np.sqrt(mix.covariances_.flatten())

# Find the Gaussian properties of the major weight
MajorW = np.array(pi).argmax()
GausNegMean = mu[MajorW]
GausNegSD = sigma[MajorW]
NbSD = 3
GausThr = GausNegMean+(NbSD*GausNegSD)
GausThrShort = round(GausThr,1)

grid = np.arange(np.min(RedCh), np.max(RedCh), 0.01)

Test the pValue for diferent threshold

In [ ]:
# Get Min & Max Int values
Mini = Final_Table["Red_Channel_Mean_Intensity"].min()
Mini = int(round(Mini))+1
Maxi = Final_Table["Red_Channel_Mean_Intensity"].max()
Maxi = int(round(Maxi))-1
ThrList = list(range(Mini,Maxi))
ThrNb = round(Maxi-Mini)
AllThr = pd.DataFrame(columns=['Threshold', 'pValue'])

Index = 0
# Run the Mann Whitney test for every possible threshold
for Thr in ThrList:
    Positive = Final_Table.loc[Final_Table["Red_Channel_Mean_Intensity"]>Thr,"Total_Distance"]
    Negative = Final_Table.loc[Final_Table["Red_Channel_Mean_Intensity"]<=Thr,"Total_Distance"]
    s,p = mannwhitneyu(Negative, Positive)
    AllThr.at[Index,"Threshold"]=Thr
    AllThr.at[Index,"pValue"]=p
    Index = Index+1

Find the best pValue

In [ ]:
AllThr = AllThr.astype(float)

BestpValidx = AllThr["pValue"].idxmin()
BestpVal = AllThr.at[BestpValidx,"pValue"]
BestThr = AllThr.at[BestpValidx,"Threshold"]

Display the pValue for all the possible threshold

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.ticker as tck

fig2, ax = plt.subplots(2, 2, gridspec_kw={'height_ratios': [6, 1]},figsize=(10,6))
fig2.tight_layout()
ax[0,0].plot(AllThr["Threshold"],AllThr["pValue"],'dimgrey', lw=("1.5"))

# Figure 1
ax[0,0].set_title('Variation of the pValue depending on the threshold')
ax[0,0].title.set_size(11)
ax[0,0].set_xlabel("Threshold")
ax[0,0].set_ylabel("pValue")

yMax = 0.0005
if BestpVal < 0.0005:
    yMax = BestpVal / 1.5

ax[0,0].set_xlim(Mini,Maxi)
ax[0,0].set_ylim(yMax,1.1)
ax[0,0].set_yscale("log")
ax[0,0].axhspan(0, 0.001, facecolor='greenyellow', alpha=0.5)
ax[0,0].axhspan(0.001, 0.01, facecolor='yellow', alpha=0.5)
ax[0,0].axhspan(0.01, 0.05, facecolor='lightgoldenrodyellow', alpha=1)
# Create legend of fig 1
import matplotlib.patches as mpatches
patch1 = mpatches.Patch(color='greenyellow', label='***')
patch2 = mpatches.Patch(color='yellow', label='**')
patch3 = mpatches.Patch(color='lightgoldenrodyellow', label='*')
patch4 = mpatches.Patch(color='white', label='n.s.')
all_handles = (patch4, patch3, patch2, patch1)
leg = ax[0,0].legend(handles=all_handles,loc='lower right')
ax[0,0].add_artist(leg)
# Plot best threshold
ax[0,0].axvline(x=BestThr,color="darkred", linestyle=":")

# Figure 2
ax[0,1].set_title('Gaussian unmixing of two population')
ax[0,1].title.set_size(11)
ax[0,1].set_ylabel("frequency")
ax[0,1].hist(RedCh, bins=70, density=True, alpha=0.5, color='darkred')
ax[0,1].plot(grid, mix_pdf(grid, mu, sigma, pi), color='maroon', label='Gaussian Fit')
ax[0,1].axvline(x = GausThr, color = 'k', linestyle='dashed', label = 'Threshold ('+str(GausThrShort)+')')
ax[0,1].legend(loc='upper right')
ax[0,1].set_yscale('log')
ax[0,1].set_ybound(lower=0.00001, upper=None)
ax[0,1].set_xlabel("Intensity in the Red Channel")

# Add text
ax[1,0].set_title('Best Threshold ='+ str(BestThr)+'\n Based on the lowest pValue', color="darkgreen")
ax[1,1].set_title('Cutoff value ='+ str(GausThrShort)+'\n Gaussian fitting of the negative population \n (Gaussian mean + '+str(NbSD)+' SD)', color="darkred")
ax[1,0].axis('off') 
ax[1,1].axis('off') 
fig2.tight_layout()
plt.show()

# Save fig as PDF
fig2.savefig(ExpFolder+"/Threshold_Estimation_All-Foci.pdf")

# Save fig as PNG
fig2.savefig(ExpFolder+"/Threshold_Estimation_All-Foci.png", dpi=300)

In [ ]:
# Set threshold
Threshold = BestThr
Threshold = click.prompt("Set a threshold value (default = best threshold)",type=float, default=GausThrShort)

In [ ]:
# Split the Foci distance in two tables based on the threshold
Positive = Final_Table.loc[Final_Table["Red_Channel_Mean_Intensity"]>Threshold,"Total_Distance"]
Negative = Final_Table.loc[Final_Table["Red_Channel_Mean_Intensity"]<=Threshold,"Total_Distance"]

In [ ]:
# Wilcoxon test
s,p = mannwhitneyu(Negative, Positive)

if p < 0.001:
    Stat = "***"
elif p < 0.01:
    Stat = "**"
elif p <0.05:
    Stat = "*"
else:
    Stat = "n.s."

In [ ]:
Negative = Negative.to_list()
Positive = Positive.to_list()
BoxPlot =  pd.DataFrame({'Negative': pd.Series(Negative), 'Positive': pd.Series(Positive)})

### Generate a BoxPlot figure

In [ ]:
f, ax = plt.subplots(figsize=(5.5, 7.5))

sns.boxplot(data=BoxPlot, palette="Reds", fliersize=4, linewidth=1.5)

# statistical annotation
x1, x2 = 0, 1   # Between the 2 first col
y, h, col = BoxPlot['Negative'].max() + 0.03, 0.03, 'k'

plt.axhline(y=0, linestyle=':', color='gray', zorder=0)

plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, Stat, ha='center', va='bottom', color=col, fontsize='large')

nbNeg = 'n = ' + str(len(Negative))
ax.text(0,BoxPlot['Negative'].median()+0.01,nbNeg,ha='center')
nbPos = 'n = ' + str(len(Positive))
ax.text(1,BoxPlot['Positive'].median()+0.01,nbPos,ha='center')

ax.set_ylabel("Foci distance to nucleoid surface (µm)",fontsize='large')
ax.set_xlabel("Pathogenicity island expression",fontsize='large')
ax.set_title("Foci localisation upon pathogenicity island expression",fontsize='large')
ax.title.set_size(12)

plt.figtext(0.01,0.01, f'Threshold : '+str(Threshold),color="darkred")
plt.tight_layout()
plt.show()

# Save fig as PDF
f.savefig(ExpFolder+"/BoxPlot_All-Foci.pdf")

# Save fig as PNG
f.savefig(ExpFolder+"/BoxPlot_All-Foci.png", dpi=300)